In [2]:
#!/usr/bin/env python3
#pip install  rioxarray==0.3.1
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray
import numpy as np
import geopandas as gpd
import cartopy.crs as ccrs
import rasterio
import os
import matplotlib.colors
scriptsdir = os.getcwd()
from scipy.interpolate import griddata
from functools import reduce
import xarray
import itertools
import argparse
import matplotlib.colors as mcolors


In [3]:
#old

models = ["GAM","GBM"]
taxas = ["Mammals"]

model_names = ['GFDL-ESM2M', 'IPSL-CM5A-LR', 'HadGEM2-ES', 'MIROC5']
years = ['1845', '1990', '1995', '2009', '2010', '2020', '2026', '2032', '2048', '2050',
         '2052', '2056', '2080', '2100', '2150', '2200', '2250']

for taxa in taxas:
    for model in models:
        dir_species = "/storage/scratch/users/ch21o450/data/LandClim_Output/" + model + "/" + taxa + "/EWEMBI/"
        available_file = os.listdir(dir_species)
        available_names = [x.split("_[1146].nc")[0] for x in available_file]

    species_names = available_names[:10]

    def newvalue_fun(time, model, netcdf_path_format, is_historical=False, scenario=None):
        newvalue_dict = {model_name: {} for model_name in model_names}
        sum_bin_dict = {model_name: {} for model_name in model_names}

        for model_name in model_names:
            for species_name in species_names:
                if is_historical:
                    ds = xr.open_dataset(netcdf_path_format.format(model, taxa, species_name, time), decode_times=False)
                else:
                    ds = xr.open_dataset(netcdf_path_format.format(model, taxa, model_name, scenario, species_name, time), decode_times=False)

                newvalue = ds["newvalue"]
                sum_bin = ds["sum_bin"]

                newvalue_dict[model_name][species_name] = newvalue
                sum_bin_dict[model_name][species_name] = sum_bin

        projections_dict = {}

        for species_name in species_names:
            value_list = []
            for model_name in model_names:
                value_bin = newvalue_dict[model_name][species_name]
                #value_bin = value_bin.where(value_bin > 0, 1)
                #value_bin = (value_bin > 0.00)

                value_list.append(value_bin)
            value_bin_concat = xr.concat(value_list, dim="model")
            mean_value_bin = value_bin_concat.mean(dim="model")
            projections_dict[species_name] = mean_value_bin

        value_bin_list = list(projections_dict.values())
        mean_value_bin = xr.concat(value_bin_list, dim="species").sum(dim="species")  # Ensemble mean over species
        mean_value_bin = mean_value_bin.where(mean_value_bin > 0, 0)
        return mean_value_bin

    def calculate_mean(time, model, netcdf_path_format, is_historical=False, scenario=None):
        newvalue_dict = {model_name: {} for model_name in model_names}
        sum_bin_dict = {model_name: {} for model_name in model_names}
        lu_sum_bin_dict = {model_name: {} for model_name in model_names}

        for model_name in model_names:
            for species_name in species_names:
                if is_historical:
                    ds = xr.open_dataset(netcdf_path_format.format(model, taxa, species_name, time), decode_times=False)
                else:
                    ds = xr.open_dataset(netcdf_path_format.format(model, taxa, model_name, scenario, species_name, time), decode_times=False)
                sum_bin = ds["sum_bin"]
                #lu_sum_bin = ds["sum_lu_binary"]
                #sum_bin = (sum_bin > 0.00)

                sum_bin_dict[model_name][species_name] = sum_bin
                #lu_sum_bin_dict[model_name][species_name] = lu_sum_bin

        projections_dict = {}

        for species_name in species_names:
            sum_bin_list = []
            for model_name in model_names:
                sum_bin = sum_bin_dict[model_name][species_name]
                sum_bin_list.append(sum_bin)
            sum_bin_concat = xr.concat(sum_bin_list, dim="model")
            mean_sum_bin = sum_bin_concat.mean(dim="model")
            projections_dict[species_name] = mean_sum_bin

        mean_sum_bin_list = list(projections_dict.values())
        mean_sum_bin = xr.concat(mean_sum_bin_list, dim="species").sum(dim="species")  # Ensemble mean over species
        mean_sum_bin = mean_sum_bin.where(mean_sum_bin > 0, 0)

        return mean_sum_bin

    historical_time = 1146
    future_times = [35, 65]
    scenarios = ["rcp26"]

    netcdf_path_format_future = "/storage/scratch/users/ch21o450/data/LandClim_Output/{}/{}/{}/{}/{}_[{}].nc"
    netcdf_path_format_hist = "/storage/scratch/users/ch21o450/data/LandClim_Output/{}/{}/EWEMBI/{}_[{}].nc"

    mean_value_bin_hist = newvalue_fun(historical_time, model, netcdf_path_format_hist, is_historical=True)
    mean_sum_bin_hist = calculate_mean(historical_time, model, netcdf_path_format_hist, is_historical=True)

    mean_sum_bin_hist = mean_sum_bin_hist.isel(time=0)

    year_indices = {1146: '1995', 35: '2050', 65: '2080', 85: '2100'}

    for future_time in future_times:
        for scenario in scenarios:
            if future_time == 35 or future_time == 65:
                model_names = ['GFDL-ESM2M', 'IPSL-CM5A-LR', 'HadGEM2-ES', 'MIROC5']
            elif future_time == 85:
                model_names = ['IPSL-CM5A-LR', 'HadGEM2-ES', 'MIROC5']

            mean_value_bin_future = newvalue_fun(future_time, model, netcdf_path_format_future, is_historical=False, scenario=scenario)
            mean_sum_bin_future = calculate_mean(future_time, model, netcdf_path_format_future, is_historical=False, scenario=scenario)
            mean_sum_bin_future = mean_sum_bin_future.isel(time=0)




In [28]:
netcdf_path_format_future = "/storage/scratch/users/ch21o450/data/LandClim_Output/{}/{}/{}/{}/{}_[{}].nc"
model_names = ['GFDL-ESM2M', 'IPSL-CM5A-LR', 'HadGEM2-ES', 'MIROC5']
years = ['1845', '1990', '1995', '2009', '2010', '2020', '2026', '2032', '2048', '2050',
         '2052', '2056', '2080', '2100', '2150', '2200', '2250']
time=35
models=["GAM"]
for taxa in taxas:
    for model in models:
        dir_species = "/storage/scratch/users/ch21o450/data/LandClim_Output/" + model + "/" + taxa + "/EWEMBI/"
        available_file = os.listdir(dir_species)
        available_names = [x.split("_[1146].nc")[0] for x in available_file]

    species_names = available_names[:10]

    for model_name in model_names:
        for species_name in species_names:


            filename = f"/storage/homefs/ch21o450/scripts/BioScenComb/intermediate_results/{taxa}_{model}_{future_time}_{scenario}_species_count_per_SCDM_05th_mean_value_bin_future.nc"

            filename2 = f"/storage/homefs/ch21o450/scripts/BioScenComb/intermediate_results/{taxa}_{model}_{future_time}_{scenario}_species_count_per_SCDM_05th_mean_sum_bin_future.nc"

            mean_value_bin_future = xr.open_dataset(filename,decode_times=False).to_array()
            mean_sum_bin_future = xr.open_dataset(filename2,decode_times=False).to_array()

In [16]:
sum_bin_concat

NameError: name 'sum_bin_concat' is not defined

In [43]:
newvalue_dict = {model_name: {} for model_name in model_names}
sum_bin_dict = {model_name: {} for model_name in model_names}
lu_sum_bin_dict = {model_name: {} for model_name in model_names}

for model_name in model_names:
    for species_name in species_names:

        ds = xr.open_dataset(netcdf_path_format_future.format(model, taxa, model_name, scenario, species_name, time), decode_times=False)
        sum_bin = ds["sum_bin"]
        #lu_sum_bin = ds["sum_lu_binary"]
        #sum_bin = (sum_bin > 0.00)

        sum_bin_dict[model_name][species_name] = sum_bin
        #lu_sum_bin_dict[model_name][species_name] = lu_sum_bin

projections_dict = {}

for species_name in species_names:
    sum_bin_list = []
    for model_name in model_names:
        sum_bin = sum_bin_dict[model_name][species_name]
        sum_bin_list.append(sum_bin)
    sum_bin_concat = xr.concat(sum_bin_list, dim="model_name")
    mean_sum_bin = sum_bin_concat.mean(dim="model_name")
    projections_dict[species_name] = mean_sum_bin

mean_sum_bin_list = list(projections_dict.values())
mean_sum_bin = xr.concat(mean_sum_bin_list, dim="species").sum(dim="species")  # Ensemble mean over species
mean_sum_bin = mean_sum_bin.where(mean_sum_bin > 0, 0)

In [44]:
mean_sum_bin.mean()

<xarray.DataArray 'sum_bin' ()>
array(0.02418364)

In [42]:
mean_sum_bin.mean()

<xarray.DataArray 'sum_bin' ()>
array(0.02418364)

In [39]:
sum_bin_concat.mean()

<xarray.DataArray 'sum_bin' ()>
array(0.00747423)

In [37]:
sum_bin_concat.mean()

<xarray.DataArray 'sum_bin' ()>
array(0.00747423)

In [35]:
sum_bin_list

[<xarray.DataArray 'sum_bin' (time: 1, lat: 360, lon: 720)>
 [259200 values with dtype=float64]
 Coordinates:
   * time     (time) float64 35.0
   * lon      (lon) float64 -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8
   * lat      (lat) float64 89.75 89.25 88.75 88.25 ... -88.75 -89.25 -89.75,
 <xarray.DataArray 'sum_bin' (time: 1, lat: 360, lon: 720)>
 [259200 values with dtype=float64]
 Coordinates:
   * time     (time) float64 35.0
   * lon      (lon) float64 -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8
   * lat      (lat) float64 89.75 89.25 88.75 88.25 ... -88.75 -89.25 -89.75,
 <xarray.DataArray 'sum_bin' (time: 1, lat: 360, lon: 720)>
 [259200 values with dtype=float64]
 Coordinates:
   * time     (time) float64 35.0
   * lon      (lon) float64 -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8
   * lat      (lat) float64 89.75 89.25 88.75 88.25 ... -88.75 -89.25 -89.75,
 <xarray.DataArray 'sum_bin' (time: 1, lat: 360, lon: 720)>
 [259200 values with dtype=float64]
 Coo

In [8]:
#new


scenarios = ["rcp26"]


model_names = ['GFDL-ESM2M', 'IPSL-CM5A-LR', 'HadGEM2-ES', 'MIROC5']
years = ['1845', '1990', '1995', '2009', '2010', '2020', '2026', '2032', '2048', '2050',
         '2052', '2056', '2080', '2100', '2150', '2200', '2250']

for taxa in taxas:
    for model in models:
        dir_species = "/storage/scratch/users/ch21o450/data/LandClim_Output/" + model + "/" + taxa + "/EWEMBI/"
        available_file = os.listdir(dir_species)
        available_names = [x.split("_[1146].nc")[0] for x in available_file]

    species_names = available_names[:10]

    def newvalue_fun(time, model, netcdf_path_format, is_historical=False, scenario=None):
        newvalue_dict = {model_name: {} for model_name in model_names}
        sum_bin_dict = {model_name: {} for model_name in model_names}

        for model_name in model_names:
            for species_name in species_names:
                if is_historical:
                    ds = xr.open_dataset(netcdf_path_format.format(model, taxa, species_name, time), decode_times=False)
                else:
                    ds = xr.open_dataset(netcdf_path_format.format(model, taxa, model_name, scenario, species_name, time), decode_times=False)

                newvalue = ds["newvalue"]
                sum_bin = ds["sum_bin"]

                newvalue_dict[model_name][species_name] = newvalue
                sum_bin_dict[model_name][species_name] = sum_bin

        projections_dict = {}

        for species_name in species_names:
            value_list = []
            for model_name in model_names:
                value_bin = newvalue_dict[model_name][species_name]
                #value_bin = value_bin.where(value_bin > 0, 1)
                #value_bin = (value_bin > 0.00)

                value_list.append(value_bin)
            value_bin_concat = xr.concat(value_list, dim="model_name")
            mean_value_bin = value_bin_concat.mean(dim="model_name")
            projections_dict[species_name] = mean_value_bin

        value_bin_list = list(projections_dict.values())
        mean_value_bin = xr.concat(value_bin_list, dim="species").sum(dim="species")  # Ensemble mean over species
        mean_value_bin = mean_value_bin.where(mean_value_bin > 0, 0)
        return mean_value_bin

    def calculate_mean(time, model, netcdf_path_format, is_historical=False, scenario=None):
        newvalue_dict = {model_name: {} for model_name in model_names}
        sum_bin_dict = {model_name: {} for model_name in model_names}
        lu_sum_bin_dict = {model_name: {} for model_name in model_names}

        for model_name in model_names:
            for species_name in species_names:
                if is_historical:
                    ds = xr.open_dataset(netcdf_path_format.format(model, taxa, species_name, time), decode_times=False)
                else:
                    ds = xr.open_dataset(netcdf_path_format.format(model, taxa, model_name, scenario, species_name, time), decode_times=False)
                sum_bin = ds["sum_bin"]
                #lu_sum_bin = ds["sum_lu_binary"]
                #sum_bin = (sum_bin > 0.00)

                sum_bin_dict[model_name][species_name] = sum_bin
                #lu_sum_bin_dict[model_name][species_name] = lu_sum_bin

        projections_dict = {}

        for species_name in species_names:
            sum_bin_list = []
            for model_name in model_names:
                sum_bin = sum_bin_dict[model_name][species_name]
                sum_bin_list.append(sum_bin)
            sum_bin_concat = xr.concat(sum_bin_list, dim="model_name")
            mean_sum_bin = sum_bin_concat.mean(dim="model_name")
            projections_dict[species_name] = mean_sum_bin

        mean_sum_bin_list = list(projections_dict.values())
        mean_sum_bin = xr.concat(mean_sum_bin_list, dim="species").sum(dim="species")  # Ensemble mean over species
        mean_sum_bin = mean_sum_bin.where(mean_sum_bin > 0, 0)

        return mean_sum_bin

    historical_time = 1146
    future_times = [35, 65]
   # scenarios = ["rcp26"]

    netcdf_path_format_future = "/storage/scratch/users/ch21o450/data/LandClim_Output/{}/{}/{}/{}/{}_[{}].nc"
    netcdf_path_format_hist = "/storage/scratch/users/ch21o450/data/LandClim_Output/{}/{}/EWEMBI/{}_[{}].nc"

    mean_value_bin_hist_new = newvalue_fun(historical_time, model, netcdf_path_format_hist, is_historical=True)
    mean_sum_bin_hist_new = calculate_mean(historical_time, model, netcdf_path_format_hist, is_historical=True)

    mean_sum_bin_hist_new = mean_sum_bin_hist_new.isel(time=0)

    year_indices = {1146: '1995', 35: '2050', 65: '2080', 85: '2100'}

    for future_time in future_times:
        for scenario in scenarios:
            if future_time == 35 or future_time == 65:
                model_names = ['GFDL-ESM2M', 'IPSL-CM5A-LR', 'HadGEM2-ES', 'MIROC5']
            elif future_time == 85:
                model_names = ['IPSL-CM5A-LR', 'HadGEM2-ES', 'MIROC5']



            mean_value_bin_future_new = newvalue_fun(future_time, model, netcdf_path_format_future, is_historical=False, scenario=scenario)
            mean_sum_bin_future_new = calculate_mean(future_time, model, netcdf_path_format_future, is_historical=False, scenario=scenario)
            mean_sum_bin_future_new = mean_sum_bin_future_new.isel(time=0)

